In [206]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [207]:
def squared_euclidean_distance(x_1,x_2):
    sqd = np.sqrt(np.reshape(np.sum(x_1**2,1),newshape =(x_1.shape[0],1))+\
    np.reshape(np.sum(x_2**2,1),newshape =(1,x_2.shape[0]))-\
    2*(x_1@x_2.T))
    return sqd

def cartesian_dist(x_1,x_2):
    return np.concatenate([
        np.tile(x_1[:,0] - np.reshape(x_2[:,0],[x_2[:,0].shape[0],1]),[1,3]),
        np.tile(x_1[:,1] - np.reshape(x_2[:,1],[x_2[:,1].shape[0],1]),[1,3]),
        np.tile(x_1[:,2] - np.reshape(x_2[:,2],[x_2[:,2].shape[0],1]),[1,3])],axis = 0) 

def cov_gradients(dist_tiled,h_u,h_v,a_T,perpendicularity_matrix):
    c_o_T = a_T**2/14/3
    condition1 = 0
    a = (h_u*h_v)
    b = dist_tiled**2

    t1 =  np.divide(a, b, out=np.zeros_like(a),casting='unsafe', where=b!=0)
    t2 = np.where(dist_tiled < a_T,(-c_o_T*((-14/a_T**2)+
                 105/4*dist_tiled/a_T**3 -
                 35/2 * dist_tiled**3 / a_T **5 +
                 21 /4 * dist_tiled**5/a_T**7)),0)+\
          np.where(dist_tiled < a_T,(c_o_T * 7 * (9 * dist_tiled ** 5 -
                      20 * a_T ** 2 * dist_tiled ** 3 +
                      15 * a_T ** 4 * dist_tiled -
                      4 * a_T ** 5) / (2 * a_T ** 7)),0)

    # when we do the covariance of Gx with Gx, Gy with Gy and so on, quation B9 in Gempy paper
    t3 = perpendicularity_matrix * \
          np.where(dist_tiled < a_T,(c_o_T * ((-14 / a_T ** 2) + 105 / 4 * dist_tiled / a_T ** 3 -
                   35 / 2 * dist_tiled ** 3 / a_T ** 5 +
                   21 / 4 * dist_tiled ** 5 / a_T ** 7)),0)
    t4 = 1/3*np.eye(dist_tiled.shape[0])

    condition2 = t1 * t2 - t3 + t4

    C_G = np.where(dist_tiled==0, condition1, condition2) ## adding nugget effect
    return C_G

    C_G = np.where(dist_tiled==0, condition1, condition2) ## adding nugget effect
    return C_G

def set_rest_ref_matrix(number_of_points_per_surface,layer1):
    ref_layer_points = np.repeat(np.stack([layer1[-1]],axis = 0),repeats=number_of_points_per_surface-1,axis = 0)
    rest_layer_points = np.concatenate([layer1[0:-1]],axis = 0)
    return ref_layer_points,rest_layer_points

def cov_interface(ref_layer_points,rest_layer_points,a_T):
    sed_rest_rest = squared_euclidean_distance(rest_layer_points,rest_layer_points)
    sed_ref_rest = squared_euclidean_distance(ref_layer_points,rest_layer_points)
    sed_rest_ref = squared_euclidean_distance(rest_layer_points,ref_layer_points)
    sed_ref_ref = squared_euclidean_distance(ref_layer_points,ref_layer_points)
    c_o_T = a_T**2/14/3
    C_I = c_o_T*(\
                (np.where(sed_rest_rest < a_T, (1 - 7 * (sed_rest_rest / a_T) ** 2 +\
                 35 / 4 * (sed_rest_rest / a_T) ** 3 -\
                 7 / 2 * (sed_rest_rest / a_T) ** 5 +\
                 3 / 4 * (sed_rest_rest / a_T) ** 7),0)) -\
                (np.where(sed_ref_rest < a_T, (1 - 7 * (sed_ref_rest / a_T) ** 2 +\
                  35 / 4 * (sed_ref_rest / a_T) ** 3 -\
                  7 / 2 * (sed_ref_rest / a_T) ** 5 +\
                  3 / 4 * (sed_ref_rest / a_T) ** 7),0)) -\
                (np.where(sed_rest_ref < a_T, (1 - 7 * (sed_rest_ref / a_T) ** 2 +\
                  35 / 4 * (sed_rest_ref / a_T) ** 3 -\
                  7 / 2 * (sed_rest_ref / a_T) ** 5 +\
                  3 / 4 * (sed_rest_ref / a_T) ** 7),0)) +\
                 (np.where(sed_ref_ref < a_T, (1 - 7 * (sed_ref_ref / a_T) ** 2 +\
                  35 / 4 * (sed_ref_ref / a_T) ** 3 -\
                  7 / 2 * (sed_ref_ref / a_T) ** 5 +\
                  3 / 4 * (sed_ref_ref / a_T) ** 7),0)))
    
    return C_I

def cartesian_dist_no_tile(x_1,x_2):
    return np.concatenate([
        np.transpose((x_1[:,0] - np.reshape(x_2[:,0],[x_2.shape[0],1]))),
        np.transpose((x_1[:,1] - np.reshape(x_2[:,1],[x_2.shape[0],1]))),
        np.transpose((x_1[:,2] - np.reshape(x_2[:,2],[x_2.shape[0],1])))],axis = 0) 

def cov_interface_gradients(hu_rest,hu_ref,sed_dips_rest,sed_dips_ref,a_T):
    c_o_T = a_T**2/14/3
    C_GI = hu_rest*np.where(sed_dips_rest < a_T,(- c_o_T * ((-14 / a_T ** 2) + 105 / 4 * sed_dips_rest / a_T ** 3 -
                                  35 / 2 * sed_dips_rest ** 3 / a_T ** 5 +
                                  21 / 4 * sed_dips_rest ** 5 / a_T ** 7)),0)-\
    hu_ref*np.where(sed_dips_ref < a_T,(-c_o_T * ((-14 / a_T ** 2) + 105 / 4 * sed_dips_ref / a_T ** 3 -
                                  35 / 2 * sed_dips_ref ** 3 / a_T ** 5 +
                                  21 / 4 * sed_dips_ref ** 5 / a_T ** 7)),0)
    return C_GI

def safe_arange(start, stop, step):
    return step * np.arange(start / step, stop / step)

def perpendicularity(G_1):
    a = np.concatenate([np.ones([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]])],axis = 1)
    b = np.concatenate([np.zeros([G_1.shape[0],G_1.shape[0]]),np.ones([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]])],axis = 1)
    c = np.concatenate([np.zeros([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]]),np.ones([G_1.shape[0],G_1.shape[0]])],axis = 1)
    return np.concatenate([a,b,c],axis = 0)    



In [208]:
G_t = np.array([[0,10,0],[13,3,0]])
G_t_o = np.array([[0,1,0],[-1,1.7,0]])
r_t = 20
layer_t = np.array([[2,8,0],[6,7,0],[13,6,0],[18,8,0]])



G_d = np.array([[0,0,10],[0,13,3]])
G_d_o = np.array([[0,0,1],[0,-1,-1.7]])
r_d = 20
layer_d = np.array([[0,8,2],[0,7,6],[0,6,13],[0,8,18]])

G_t = G_t

G_t_o = G_t_o

## defining interface data
layer_t = layer_t

## deining range
a_T_t = r_t

c_o_T = a_T_t**2/14/3

G_t_tiled = np.tile(G_t,[3,1])

h_u_t = cartesian_dist(G_t,G_t)
h_v_t = h_u_t.T

perpendicularity_matrix_t = perpendicularity(G_t)

dist_tiled_t = squared_euclidean_distance(G_t_tiled,G_t_tiled)

dist_tiled_t = dist_tiled_t + np.eye(dist_tiled_t.shape[0])

C_G_t = cov_gradients(dist_tiled_t,h_u_t,h_v_t,a_T_t,perpendicularity_matrix_t)

number_of_points_per_surface_t = np.array([layer_t.shape[0]])

ref_layer_points_t,rest_layer_points_t = set_rest_ref_matrix(number_of_points_per_surface_t,layer_t)





# input data

## defining the dips position
G_d = G_d

G_d_o = G_d_o

## defining interface data
layer_d = layer_d

## deining range
a_T_d = r_d

c_o_T = a_T_d**2/14/3

G_d_tiled = np.tile(G_d,[3,1])

h_u_d = cartesian_dist(G_d,G_d)
h_v_d = h_u_d.T

perpendicularity_matrix_d = perpendicularity(G_d)

dist_tiled_d = squared_euclidean_distance(G_d_tiled,G_d_tiled)

dist_tiled_d = dist_tiled_d + np.eye(dist_tiled_d.shape[0])

C_G_d = cov_gradients(dist_tiled_d,h_u_d,h_v_d,a_T_d,perpendicularity_matrix_d)

number_of_points_per_surface_d = np.array([layer_d.shape[0]])

ref_layer_points_d,rest_layer_points_d = set_rest_ref_matrix(number_of_points_per_surface_d,layer_d)

sed_rest_rest_d = squared_euclidean_distance(rest_layer_points_d,rest_layer_points_d)
sed_ref_rest_d = squared_euclidean_distance(ref_layer_points_d,rest_layer_points_d)
sed_rest_ref_d = squared_euclidean_distance(rest_layer_points_d,ref_layer_points_d)
sed_ref_ref_d = squared_euclidean_distance(ref_layer_points_d,ref_layer_points_d)

C_I_d = cov_interface(ref_layer_points_d,rest_layer_points_d,a_T_d)

sed_dips_rest_d = squared_euclidean_distance(G_d_tiled,rest_layer_points_d)
sed_dips_ref_d = squared_euclidean_distance(G_d_tiled,ref_layer_points_d)

hu_rest_d = cartesian_dist_no_tile(G_d,rest_layer_points_d)
hu_ref_d = cartesian_dist_no_tile(G_d,ref_layer_points_d)

C_GI_d = cov_interface_gradients(hu_rest_d,hu_ref_d,sed_dips_rest_d,sed_dips_ref_d,a_T_d)
C_IG_d = C_GI_d.T

xx = safe_arange(0,20,1)
yy = safe_arange(0,20,1)
zz = safe_arange(0,20,1)
XX,YY,ZZ = np.meshgrid(xx,yy,zz)
X = (np.reshape(XX,[-1])).T
Y = (np.reshape(YY,[-1])).T
Z = (np.reshape(ZZ,[-1])).T
X = np.zeros(X.shape)
Y = np.zeros(Y.shape)
grid_d = np.stack([X,Y,Z],axis = 1)


hu_Simpoints_d = cartesian_dist_no_tile(G_d,grid_d)
sed_dips_SimPoint_d = squared_euclidean_distance(G_d_tiled,grid_d)
sed_rest_SimPoint_d = squared_euclidean_distance(rest_layer_points_d,grid_d)
sed_ref_SimPoint_d = squared_euclidean_distance(ref_layer_points_d,grid_d)


K_d = np.concatenate([np.concatenate([C_G_d,C_GI_d],axis = 1),np.concatenate([C_IG_d,C_I_d],axis = 1)],axis = 0)

a1 = np.concatenate((np.ones(G_d.shape[0]),np.zeros(G_d.shape[0]),np.zeros(G_d.shape[0])))
a2 = np.concatenate((np.zeros(G_d.shape[0]),np.ones(G_d.shape[0]),np.zeros(G_d.shape[0])))
a3 = np.concatenate((np.zeros(G_d.shape[0]),np.zeros(G_d.shape[0]),np.ones(G_d.shape[0])))
a3 = np.vstack([a1,a2,a3])

a4 = np.stack([ref_layer_points_d[:,0] - rest_layer_points_d[:,0],\
                ref_layer_points_d[:,1] - rest_layer_points_d[:,1],\
                ref_layer_points_d[:,2] - rest_layer_points_d[:,2]])


a51 = np.concatenate((2*G_d[:,0],np.zeros(G_d.shape[0]),np.zeros(G_d.shape[0])))

a52 = np.concatenate((np.zeros(G_d.shape[0]),2*G_d[:,1],np.zeros(G_d.shape[0])))

a53 = np.concatenate((np.zeros(G_d.shape[0]),np.zeros(G_d.shape[0]),2*G_d[:,2]))

a54 = np.concatenate((G_d[:,1],G_d[:,0],np.zeros(G_d.shape[0])))

a55 = np.concatenate((G_d[:,2],np.zeros(G_d.shape[0]),G_d[:,0]))

a56 = np.concatenate((np.zeros(G_d.shape[0]),G_d[:,2],G_d[:,1]))

a5 = np.vstack([a51,a52,a53,a54,a55,a56])

a6 = np.stack([ref_layer_points_d[:,0]*ref_layer_points_d[:,0].T - rest_layer_points_d[:,0]*rest_layer_points_d[:,0].T , \
            ref_layer_points_d[:,1]*ref_layer_points_d[:,1].T - rest_layer_points_d[:,1]*rest_layer_points_d[:,1].T , \
            ref_layer_points_d[:,2]*ref_layer_points_d[:,2].T - rest_layer_points_d[:,2]*rest_layer_points_d[:,2].T, \
            ref_layer_points_d[:,0]*ref_layer_points_d[:,1] - rest_layer_points_d[:,0]*rest_layer_points_d[:,1],\
            ref_layer_points_d[:,0]*ref_layer_points_d[:,2] - rest_layer_points_d[:,0]*rest_layer_points_d[:,2],\
            ref_layer_points_d[:,1]*ref_layer_points_d[:,2] - rest_layer_points_d[:,1]*rest_layer_points_d[:,2]])



D_d = np.concatenate([np.concatenate([a3,a4],axis=1),np.concatenate([a5,a6],axis=1)], axis=0)
D_d_T = D_d.T

zero_matrix_d = np.zeros([D_d_T.shape[1],D_d_T.shape[1]])
K_D_d = np.concatenate([np.concatenate([K_d,D_d_T],axis = 1),np.concatenate([D_d,zero_matrix_d],axis = 1)],axis = 0)

# build right side matrix of cokriging system
bk_d = np.concatenate([G_d_o[:,0],G_d_o[:,1],G_d_o[:,2],np.zeros(K_D_d.shape[0]-G_d.shape[0]*3)],axis = 0)
bk_d = np.reshape(bk_d,newshape = [bk_d.shape[0],1])

# solve kriging weight
w_d = np.linalg.lstsq(K_D_d,bk_d)[0]

#######################################################
grid_r=ref_layer_points_t
# grid_r[:,0] = np.zeros(grid_r[:,0].shape)
# grid_r[:,1] = np.zeros(grid_r[:,0].shape)
grid_s=rest_layer_points_t
# grid_s[:,0] = np.zeros(grid_s[:,0].shape)
# grid_s[:,1] = np.zeros(grid_s[:,0].shape)

hu_Simpoints_s = cartesian_dist_no_tile(G_d,grid_s)
sed_dips_SimPoint_s = squared_euclidean_distance(G_d_tiled,grid_s)
sed_rest_SimPoint_s = squared_euclidean_distance(rest_layer_points_d,grid_s)
sed_ref_SimPoint_s = squared_euclidean_distance(ref_layer_points_d,grid_s)

hu_Simpoints_r = cartesian_dist_no_tile(G_d,grid_r)
sed_dips_SimPoint_r = squared_euclidean_distance(G_d_tiled,grid_r)
sed_rest_SimPoint_r = squared_euclidean_distance(rest_layer_points_d,grid_r)
sed_ref_SimPoint_r = squared_euclidean_distance(ref_layer_points_d,grid_r)

# gradient contribution
sigma_0_grad_r = w_d[:G_d.shape[0]*3] * (-hu_Simpoints_r*(sed_dips_SimPoint_r < a_T_d)*(- c_o_T * ((-14 / a_T_d ** 2) + 
                    105 / 4 * sed_dips_SimPoint_r / a_T_d ** 3 - 
                    35 / 2 * sed_dips_SimPoint_r ** 3 / a_T_d ** 5 + 
                    21 / 4 * sed_dips_SimPoint_r ** 5 / a_T_d ** 7)))

sigma_0_grad_r = np.sum(sigma_0_grad_r,axis=0)
# surface point contribution
sigma_0_interf_r = -w_d[G_d.shape[0]*3:-9]*(c_o_T  * ((sed_rest_SimPoint_r < a_T_d)*(1 - 7 * (sed_rest_SimPoint_r / a_T_d) ** 2 +
            35 / 4 * (sed_rest_SimPoint_r / a_T_d) ** 3 -
            7 / 2 * (sed_rest_SimPoint_r / a_T_d) ** 5 +
            3 / 4 * (sed_rest_SimPoint_r / a_T_d) ** 7) -
            (sed_ref_SimPoint_r < a_T_d)*(1 - 7 * (sed_ref_SimPoint_r / a_T_d) ** 2 +
            35 / 4 * (sed_ref_SimPoint_r / a_T_d) ** 3 -
            7 / 2 * (sed_ref_SimPoint_r / a_T_d) ** 5 +
            3 / 4 * (sed_ref_SimPoint_r / a_T_d) ** 7)))
sigma_0_interf_r = np.sum(sigma_0_interf_r,axis = 0)

# 2nd order drift contribution
sigma_0_2nd_drift_1 = np.sum(grid_r* (w_d[-9:-6]).T,axis = 1)

sigma_0_2nd_drift_xx = grid_r[:,0]*grid_r[:,0]* (w_d[-6]).T
sigma_0_2nd_drift_yy = grid_r[:,1]*grid_r[:,1]* (w_d[-5]).T
sigma_0_2nd_drift_zz = grid_r[:,2]*grid_r[:,2]* (w_d[-4]).T

sigma_0_2nd_drift_xy = grid_r[:,0]*grid_r[:,1]* (w_d[-3]).T
sigma_0_2nd_drift_xz = grid_r[:,0]*grid_r[:,2]* (w_d[-2]).T
sigma_0_2nd_drift_yz = grid_r[:,1]*grid_r[:,2]* (w_d[-1]).T

sigma_0_2nd_drift_r = sigma_0_2nd_drift_1 + sigma_0_2nd_drift_xx + sigma_0_2nd_drift_yy + sigma_0_2nd_drift_zz + sigma_0_2nd_drift_xy + sigma_0_2nd_drift_xz + sigma_0_2nd_drift_yz

interpolate_result_r = sigma_0_grad_r+sigma_0_interf_r+sigma_0_2nd_drift_r

# gradient contribution
sigma_0_grad_s = w_d[:G_d.shape[0]*3] * (-hu_Simpoints_s*(sed_dips_SimPoint_s < a_T_d)*(- c_o_T * ((-14 / a_T_d ** 2) + 
                    105 / 4 * sed_dips_SimPoint_s / a_T_d ** 3 - 
                    35 / 2 * sed_dips_SimPoint_s ** 3 / a_T_d ** 5 + 
                    21 / 4 * sed_dips_SimPoint_s ** 5 / a_T_d ** 7)))

sigma_0_grad_s = np.sum(sigma_0_grad_s,axis=0)
# surface point contribution
sigma_0_interf_s = -w_d[G_d.shape[0]*3:-9]*(c_o_T  * ((sed_rest_SimPoint_s < a_T_d)*(1 - 7 * (sed_rest_SimPoint_s / a_T_d) ** 2 +
            35 / 4 * (sed_rest_SimPoint_s / a_T_d) ** 3 -
            7 / 2 * (sed_rest_SimPoint_s / a_T_d) ** 5 +
            3 / 4 * (sed_rest_SimPoint_s / a_T_d) ** 7) -
            (sed_ref_SimPoint_s < a_T_d)*(1 - 7 * (sed_ref_SimPoint_s / a_T_d) ** 2 +
            35 / 4 * (sed_ref_SimPoint_s / a_T_d) ** 3 -
            7 / 2 * (sed_ref_SimPoint_s / a_T_d) ** 5 +
            3 / 4 * (sed_ref_SimPoint_s / a_T_d) ** 7)))
sigma_0_interf_s = np.sum(sigma_0_interf_s,axis = 0)

# 2nd order drift contribution
sigma_0_2nd_drift_1 = np.sum(grid_s* (w_d[-9:-6]).T,axis = 1)

sigma_0_2nd_drift_xx = grid_s[:,0]*grid_s[:,0]* (w_d[-6]).T
sigma_0_2nd_drift_yy = grid_s[:,1]*grid_s[:,1]* (w_d[-5]).T
sigma_0_2nd_drift_zz = grid_s[:,2]*grid_s[:,2]* (w_d[-4]).T

sigma_0_2nd_drift_xy = grid_s[:,0]*grid_s[:,1]* (w_d[-3]).T
sigma_0_2nd_drift_xz = grid_s[:,0]*grid_s[:,2]* (w_d[-2]).T
sigma_0_2nd_drift_yz = grid_s[:,1]*grid_s[:,2]* (w_d[-1]).T

sigma_0_2nd_drift_s = sigma_0_2nd_drift_1 + sigma_0_2nd_drift_xx + sigma_0_2nd_drift_yy + sigma_0_2nd_drift_zz + sigma_0_2nd_drift_xy + sigma_0_2nd_drift_xz + sigma_0_2nd_drift_yz

interpolate_result_s = sigma_0_grad_s+sigma_0_interf_s+sigma_0_2nd_drift_s

final_result = interpolate_result_r - interpolate_result_s
print(final_result)
#######################################################


# gradient contribution
sigma_0_grad_d = w_d[:G_d.shape[0]*3] * (-hu_Simpoints_d*(sed_dips_SimPoint_d < a_T_d)*(- c_o_T * ((-14 / a_T_d ** 2) + 105 / 4 * sed_dips_SimPoint_d / a_T_d ** 3 - 35 / 2 * sed_dips_SimPoint_d ** 3 / a_T_d ** 5 + 21 / 4 * sed_dips_SimPoint_d ** 5 / a_T_d ** 7)))

sigma_0_grad_d = np.sum(sigma_0_grad_d,axis=0)
# surface point contribution
sigma_0_interf_d = -w_d[G_d.shape[0]*3:-9]*(c_o_T  * ((sed_rest_SimPoint_d < a_T_d)*(1 - 7 * (sed_rest_SimPoint_d / a_T_d) ** 2 +
            35 / 4 * (sed_rest_SimPoint_d / a_T_d) ** 3 -
            7 / 2 * (sed_rest_SimPoint_d / a_T_d) ** 5 +
            3 / 4 * (sed_rest_SimPoint_d / a_T_d) ** 7) -
            (sed_ref_SimPoint_d < a_T_d)*(1 - 7 * (sed_ref_SimPoint_d / a_T_d) ** 2 +
            35 / 4 * (sed_ref_SimPoint_d / a_T_d) ** 3 -
            7 / 2 * (sed_ref_SimPoint_d / a_T_d) ** 5 +
            3 / 4 * (sed_ref_SimPoint_d / a_T_d) ** 7)))
sigma_0_interf_d = np.sum(sigma_0_interf_d,axis = 0)

# 2nd order drift contribution
grid_d[:,0] = np.zeros(grid_d[:,0].shape)
grid_d[:,1] = np.zeros(grid_d[:,0].shape)

sigma_0_2nd_drift_1 = np.sum(grid_d* (w_d[-9:-6]).T,axis = 1)

sigma_0_2nd_drift_xx = grid_d[:,0]*grid_d[:,0]* (w_d[-6]).T
sigma_0_2nd_drift_yy = grid_d[:,1]*grid_d[:,1]* (w_d[-5]).T
sigma_0_2nd_drift_zz = grid_d[:,2]*grid_d[:,2]* (w_d[-4]).T

sigma_0_2nd_drift_xy = grid_d[:,0]*grid_d[:,1]* (w_d[-3]).T
sigma_0_2nd_drift_xz = grid_d[:,0]*grid_d[:,2]* (w_d[-2]).T
sigma_0_2nd_drift_yz = grid_d[:,1]*grid_d[:,2]* (w_d[-1]).T

sigma_0_2nd_drift_d = sigma_0_2nd_drift_1 + sigma_0_2nd_drift_xx + sigma_0_2nd_drift_yy + sigma_0_2nd_drift_zz + sigma_0_2nd_drift_xy + sigma_0_2nd_drift_xz + sigma_0_2nd_drift_yz


interpolate_result_drift_d = sigma_0_grad_d+sigma_0_interf_d+sigma_0_2nd_drift_d
print(interpolate_result_drift_d)


[-0.09639682  0.20843053  0.87282635]
[ 0.3142771   0.98720912  1.73124139 ... 18.15833261 20.12180441
 22.22597296]


In [209]:
# G_1 = np.array([[0,10,0],[13,3,0]])

# G_1_o = np.array([[0,1,0],[-1,1.7,0]])

# ## deining range
# a_T = 20

# layer1 = np.array([[2,8,0],[6,7,0],[13,6,0],[18,8,0]])

# c_o_T = a_T**2/14/3

# G_1_tiled = np.tile(G_1,[3,1])

# h_u = cartesian_dist(G_1,G_1)
# h_v = h_u.T

# perpendicularity_matrix = perpendicularity(G_1)

# dist_tiled = squared_euclidean_distance(G_1_tiled,G_1_tiled)

# dist_tiled = dist_tiled + np.eye(dist_tiled.shape[0])

# C_G = cov_gradients(dist_tiled)

# number_of_layer = 1

# number_of_points_per_surface = np.array([layer1.shape[0]])

# ref_layer_points,rest_layer_points = set_rest_ref_matrix(number_of_points_per_surface)

sed_rest_rest_t = squared_euclidean_distance(rest_layer_points_t,rest_layer_points_t)
sed_ref_rest_t = squared_euclidean_distance(ref_layer_points_t,rest_layer_points_t)
sed_rest_ref_t = squared_euclidean_distance(rest_layer_points_t,ref_layer_points_t)
sed_ref_ref_t = squared_euclidean_distance(ref_layer_points_t,ref_layer_points_t)

C_I_t = cov_interface(ref_layer_points_t,rest_layer_points_t,a_T_t)

sed_dips_rest_t = squared_euclidean_distance(G_t_tiled,rest_layer_points_t)
sed_dips_ref_t = squared_euclidean_distance(G_t_tiled,ref_layer_points_t)

hu_rest_t = cartesian_dist_no_tile(G_t,rest_layer_points_t)
hu_ref_t = cartesian_dist_no_tile(G_t,ref_layer_points_t)


C_GI_t = cov_interface_gradients(hu_rest_t,hu_ref_t,sed_dips_rest_t,sed_dips_ref_t,a_T_t)
C_IG_t = C_GI_t.T

xx = safe_arange(0,20,1)
yy = safe_arange(0,20,1)
zz = safe_arange(0,20,1)
XX,YY,ZZ = np.meshgrid(xx,yy,zz)
X = (np.reshape(XX,[-1])).T
Y = (np.reshape(YY,[-1])).T
Z = (np.reshape(ZZ,[-1])).T
grid_t = np.stack([X,Y,Z],axis = 1)


hu_Simpoints = cartesian_dist_no_tile(G_t,grid_t)
sed_dips_SimPoint_t = squared_euclidean_distance(G_t_tiled,grid_t)
sed_rest_SimPoint_t = squared_euclidean_distance(rest_layer_points_t,grid_t)
sed_ref_SimPoint_t = squared_euclidean_distance(ref_layer_points_t,grid_t)


K_t = np.concatenate([np.concatenate([C_G_t,C_GI_t],axis = 1),np.concatenate([C_IG_t,C_I_t],axis = 1)],axis = 0)

###########################

a1 = np.concatenate((np.ones(G_t.shape[0]),np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0])))
a2 = np.concatenate((np.zeros(G_t.shape[0]),np.ones(G_t.shape[0]),np.zeros(G_t.shape[0])))
a3 = np.concatenate((np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0])))
a3 = np.vstack([a1,a2,a3])

a4 = np.stack([ref_layer_points_t[:,0] - rest_layer_points_t[:,0],\
                ref_layer_points_t[:,1] - rest_layer_points_t[:,1],\
                np.zeros((ref_layer_points_t[:,2] - rest_layer_points_t[:,2]).shape[0])])


a51 = np.concatenate((2*G_t[:,0],np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0])))

a52 = np.concatenate((np.zeros(G_t.shape[0]),2*G_t[:,1],np.zeros(G_t.shape[0])))

a53 = np.concatenate((np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0])))

a54 = np.concatenate((G_t[:,1],G_t[:,0],np.zeros(G_t.shape[0])))

a55 = np.concatenate((G_t[:,2],np.zeros(G_t.shape[0]),np.zeros(G_t.shape[0])))

a56 = np.concatenate((np.zeros(G_t.shape[0]),G_t[:,2],np.zeros(G_t.shape[0])))

a5 = np.vstack([a51,a52,a53,a54,a55,a56])

a6 = np.stack([ref_layer_points_t[:,0]*ref_layer_points_t[:,0].T - rest_layer_points_t[:,0]*rest_layer_points_t[:,0].T , \
            ref_layer_points_t[:,1]*ref_layer_points_t[:,1].T - rest_layer_points_t[:,1]*rest_layer_points_t[:,1].T , \
            np.zeros((ref_layer_points_t[:,2]*ref_layer_points_t[:,2].T - rest_layer_points_t[:,2]*rest_layer_points_t[:,2].T).shape[0]), \
            ref_layer_points_t[:,0]*ref_layer_points_t[:,1] - rest_layer_points_t[:,0]*rest_layer_points_t[:,1],\
            np.zeros((ref_layer_points_t[:,0]*ref_layer_points_t[:,2] - rest_layer_points_t[:,0]*rest_layer_points_t[:,2]).shape[0]),\
            np.zeros((ref_layer_points_t[:,1]*ref_layer_points_t[:,2] - rest_layer_points_t[:,1]*rest_layer_points_t[:,2]).shape[0])])

D_U = np.concatenate([np.concatenate([a3,a4],axis=1),np.concatenate([a5,a6],axis=1)], axis=0)

D_U_T = D_U.T

########################################

D_X = G_d_o[0][0]*np.ones([G_d.shape[0],1])
D_Y = G_d_o[0][1]*np.ones([G_d.shape[0],1])
D_Z = G_d_o[0][2]*np.ones([G_d.shape[0],1])

D_I = final_result.reshape((layer_t.shape[0]-1,1))

D_E = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
print(D_E)
D_E_T = D_E.T


D = np.concatenate([D_U,D_E_T],axis=0)
D_T = D.T 

# build zero matrix
zero_matrix = np.zeros([D_T.shape[1],D_T.shape[1]])


# concatenate drift matrix and kriging matrix
K_D = np.concatenate([np.concatenate([K_t,D_T],axis = 1),np.concatenate([D,zero_matrix],axis = 1)],axis = 0)

# build right side matrix of cokriging system
bk = np.concatenate([G_t_o[:,0],G_t_o[:,1],G_t_o[:,2],np.zeros(K_D.shape[0]-G_t.shape[0]*3)],axis = 0)
bk = np.reshape(bk,newshape = [bk.shape[0],1])

# solve kriging weight
w = np.linalg.lstsq(K_D,bk)[0]

# gradient contribution
sigma_0_grad = w[:G_t.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint_t < a_T_t)*(- c_o_T * ((-14 / a_T_t ** 2) + 105 / 4 * sed_dips_SimPoint_t / a_T_t ** 3 - 35 / 2 * sed_dips_SimPoint_t ** 3 / a_T_t ** 5 + 21 / 4 * sed_dips_SimPoint_t ** 5 / a_T_t ** 7)))

sigma_0_grad = np.sum(sigma_0_grad,axis=0)
# surface point contribution
sigma_0_interf = -w[G_t.shape[0]*3:-10]*(c_o_T  * ((sed_rest_SimPoint_t < a_T_t)*(1 - 7 * (sed_rest_SimPoint_t / a_T_t) ** 2 +
            35 / 4 * (sed_rest_SimPoint_t / a_T_t) ** 3 -
            7 / 2 * (sed_rest_SimPoint_t / a_T_t) ** 5 +
            3 / 4 * (sed_rest_SimPoint_t / a_T_t) ** 7) -
            (sed_ref_SimPoint_t < a_T_t)*(1 - 7 * (sed_ref_SimPoint_t / a_T_t) ** 2 +
            35 / 4 * (sed_ref_SimPoint_t / a_T_t) ** 3 -
            7 / 2 * (sed_ref_SimPoint_t / a_T_t) ** 5 +
            3 / 4 * (sed_ref_SimPoint_t / a_T_t) ** 7)))
sigma_0_interf = np.sum(sigma_0_interf,axis = 0)

# 2nd order drift contribution
sigma_0_2nd_drift_1 = np.sum(grid_t* (w[-10:-7]).T,axis = 1)

sigma_0_2nd_drift_xx = grid_t[:,0]*grid_t[:,0]* (w[-7]).T
sigma_0_2nd_drift_yy = grid_t[:,1]*grid_t[:,1]* (w[-6]).T
sigma_0_2nd_drift_zz = grid_t[:,2]*grid_t[:,2]* (w[-5]).T

sigma_0_2nd_drift_xy = grid_t[:,0]*grid_t[:,1]* (w[-4]).T
sigma_0_2nd_drift_xz = grid_t[:,0]*grid_t[:,2]* (w[-3]).T
sigma_0_2nd_drift_yz = grid_t[:,1]*grid_t[:,2]* (w[-2]).T

sigma_0_2nd_drift = sigma_0_2nd_drift_1 + sigma_0_2nd_drift_xx + sigma_0_2nd_drift_yy + sigma_0_2nd_drift_zz + sigma_0_2nd_drift_xy + sigma_0_2nd_drift_xz + sigma_0_2nd_drift_yz

external_drift = interpolate_result_drift_d * (w[-1]).T

#external_drift = (interpolate_result_drift_d + grid_t[:,1]+ grid_t[:,0]) * (w[-1]).T

interpolate_result = 1*sigma_0_grad+1*sigma_0_interf +1*sigma_0_2nd_drift + 1 * external_drift

intp = np.reshape(interpolate_result,[20,20,20]) # reshape the result to matrix shape
intp_c = intp





[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.        ]
 [ 1.        ]
 [-0.09639682]
 [ 0.20843053]
 [ 0.87282635]]


In [210]:
import pyvista as pv
import pyvistaqt as pvqt
def plot_3D(grid,value,mode='block',surfaces_nr=10):
    x = grid[:,0]
    y = grid[:,1]
    z = grid[:,2]

    grid = pv.UniformGrid(dims=(20,20,20), spacing=(1,1,1),origin=(0,0,0))
    p = pvqt.BackgroundPlotter()
    layer1_ = layer_t
    if mode == 'block':
        p.add_mesh(grid,scalars=value.flatten(),show_edges=False,style='surface')
        p.add_axes()
        p.show()

    elif mode == 'surfaces':
        grid['vol'] = value.ravel()
        lvl_t=value[np.where((XX==layer_t[0,0])&(YY==layer_t[0,1])&(ZZ==layer_t[0,2]))]
        contours_1 = grid.contour(np.linspace(value.min(),value.max(),surfaces_nr))
        contours_2 = grid.contour(lvl_t)
        p.add_mesh(contours_1, show_scalar_bar=True, label='scalar_field',style='surface')
        p.add_mesh(contours_2, show_scalar_bar=True, label='scalar_field',style='surface',color='red')

    p.add_points(layer_t[:,[2,0,1]], render_points_as_spheres=True,
                      point_size=20.0)
    p.add_points(layer_d[:,[2,0,1]], render_points_as_spheres=True,
                      point_size=20.0 ,color='black')
    p.add_axes()
    p.show()

In [211]:
plot_3D(grid_t,intp,mode='surfaces',surfaces_nr=10)